In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp

# Load models
yolo_model = YOLO("yolov8n.pt")
gender_model = tf.keras.models.load_model("resnet82.keras")
gender_classes = ['Female', 'Male']

# Setup MediaPipe
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands
pose = mp_pose.Pose()
hands = mp_hands.Hands()
mp_draw = mp.solutions.drawing_utils

# Start webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("❌ Could not open webcam.")
    exit()

print("✅ Webcam started. Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = yolo_model(frame)
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect landmarks
    pose_result = pose.process(image_rgb)
    hand_result = hands.process(image_rgb)
    
    height, width, _ = frame.shape
    head_y = None
    hand_y_positions = []

    # Get head position (nose landmark)
    if pose_result.pose_landmarks:
        nose = pose_result.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE]
        head_y = int(nose.y * height)
        #mp_draw.draw_landmarks(frame, pose_result.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # Get hand positions
    if hand_result.multi_hand_landmarks:
        for hand_landmarks in hand_result.multi_hand_landmarks:
            for lm in hand_landmarks.landmark:
                y = int(lm.y * height)
                hand_y_positions.append(y)
            #mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    for r in results:
        for box in r.boxes:
            cls_id = int(box.cls[0])
            label = yolo_model.names[cls_id]

            if label != 'person':
                continue

            x1, y1, x2, y2 = map(int, box.xyxy[0])
            person_crop = frame[y1:y2, x1:x2]

            try:
                resized = cv2.resize(person_crop, (100, 200))
                normalized = resized / 255.0
                input_array = np.expand_dims(normalized, axis=0)

                preds = gender_model.predict(input_array, verbose=0)
                pred_class = np.argmax(preds)
                confidence = preds[0][pred_class]
                gender_label = f"{gender_classes[pred_class]} ({confidence*100:.1f}%)"

                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, gender_label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX,
                            0.6, (255, 0, 0), 2)

                # 🚨 SOS trigger check
                if gender_classes[pred_class] == "Female" and head_y and hand_y_positions:
                    for hand_y in hand_y_positions:
                        if hand_y < head_y:  # Hand above head
                            cv2.putText(frame, "SOS TRIGGERED", (50, 50),
                                        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)
                            print("SOS triggered for female raising hand")
                            break

            except Exception as e:
                print(f"⚠️ Error on crop: {e}")

    cv2.imshow("Live Gender + Gesture Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
print("✅ Webcam closed.")


✅ Webcam started. Press 'q' to quit.

0: 480x640 1 person, 1 car, 86.2ms
Speed: 1.4ms preprocess, 86.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 car, 108.1ms
Speed: 4.3ms preprocess, 108.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 car, 79.9ms
Speed: 1.1ms preprocess, 79.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 76.4ms
Speed: 1.2ms preprocess, 76.4ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 69.3ms
Speed: 2.0ms preprocess, 69.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 66.1ms
Speed: 2.1ms preprocess, 66.1ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 68.6ms
Speed: 1.4ms preprocess, 68.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 68.3ms
Speed: 2.4ms preprocess, 68.3

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp

# Load models
yolo_model = YOLO("yolov8n.pt")
gender_model = tf.keras.models.load_model("resnet82.keras")
gender_classes = ['Female', 'Male']

# Setup MediaPipe
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands
pose = mp_pose.Pose()
hands = mp_hands.Hands()
mp_draw = mp.solutions.drawing_utils

# Load video file
video_path = "testcase.mp4"  # 🔁 Replace with your actual file
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("❌ Could not open video file.")
    exit()

print("✅ Video processing started. Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = yolo_model(frame)
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect landmarks
    pose_result = pose.process(image_rgb)
    hand_result = hands.process(image_rgb)
    
    height, width, _ = frame.shape
    head_y = None
    hand_y_positions = []

    # Get head position (nose landmark)
    if pose_result.pose_landmarks:
        nose = pose_result.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE]
        head_y = int(nose.y * height)
        mp_draw.draw_landmarks(frame, pose_result.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # Get hand positions
    if hand_result.multi_hand_landmarks:
        for hand_landmarks in hand_result.multi_hand_landmarks:
            for lm in hand_landmarks.landmark:
                y = int(lm.y * height)
                hand_y_positions.append(y)
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    for r in results:
        for box in r.boxes:
            cls_id = int(box.cls[0])
            label = yolo_model.names[cls_id]

            if label != 'person':
                continue

            x1, y1, x2, y2 = map(int, box.xyxy[0])
            person_crop = frame[y1:y2, x1:x2]

            try:
                resized = cv2.resize(person_crop, (100, 200))
                normalized = resized / 255.0
                input_array = np.expand_dims(normalized, axis=0)

                preds = gender_model.predict(input_array, verbose=0)
                pred_class = np.argmax(preds)
                confidence = preds[0][pred_class]
                gender_label = f"{gender_classes[pred_class]} ({confidence*100:.1f}%)"

                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, gender_label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX,
                            0.6, (255, 0, 0), 2)

                # 🚨 SOS trigger check
                if gender_classes[pred_class] == "Female" and head_y and hand_y_positions:
                    for hand_y in hand_y_positions:
                        if hand_y < head_y:  # Hand above head
                            cv2.putText(frame, "SOS TRIGGERED", (50, 50),
                                        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)
                            print("SOS triggered for female raising hand")
                            break

            except Exception as e:
                print(f"⚠️ Error on crop: {e}")

    cv2.imshow("Gender + Gesture Detection (Video)", frame)

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
print("✅ Video processing completed.")


✅ Video processing started. Press 'q' to quit.

0: 256x640 2 persons, 1 car, 44.9ms
Speed: 2.0ms preprocess, 44.9ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 2 persons, 1 car, 50.7ms
Speed: 2.0ms preprocess, 50.7ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 2 persons, 1 car, 48.6ms
Speed: 1.0ms preprocess, 48.6ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 2 persons, 1 car, 125.4ms
Speed: 3.1ms preprocess, 125.4ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 2 persons, 1 car, 106.2ms
Speed: 3.4ms preprocess, 106.2ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 2 persons, 1 car, 57.4ms
Speed: 2.0ms preprocess, 57.4ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 2 persons, 1 car, 115.9ms
Speed: 4.2ms preprocess, 115.9ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x6